In [9]:
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
from sklearn.metrics import recall_score, accuracy_score,f1_score, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline
import warnings
import optuna
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score

In [10]:
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
raw_dataset = pd.read_csv("./data/processed_data.csv") #data has X and Y
X = raw_dataset.drop(columns=["DR"])
Y = pd.DataFrame(raw_dataset["DR"])

#* 90/10 split for training and final test
X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=random_state, stratify=Y)

Using device: cuda


Helper functions that don't need tweakin

In [11]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from modularModels1 import BlockMaker, modularNN, BasicModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)

def init_weights(model): #tested already
    if isinstance(model, nn.Linear):  # Apply only to linear layers
        nn.init.xavier_uniform_(model.weight)
        if model.bias is not None:
            nn.init.zeros_(model.bias)
            
def fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device):
    train_dataset = TensorDataset(
        torch.tensor(train_x.values,dtype=torch.float32).to(device), 
        torch.tensor(train_y.values,dtype=torch.float32).to(device))
    val_dataset = TensorDataset(
        torch.tensor(test_x.values,dtype=torch.float32).to(device), 
        torch.tensor(test_y.values,dtype=torch.float32).to(device))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
    return train_loader, val_loader 

def get_feature_count(loader):
    """returns the number of features in the dataset"""
    return next(iter(loader))[0].shape[1]

Using cuda


In [12]:
from Criterion_Models import *
def criterion_mapping(criterion_choice:str, pos_weight:float=None, alpha:float=None, gamma:float=None):
    """
    Feel free to add any custom loss functions here.
    returns function for criterion
    """
    if criterion_choice == "FocalLoss":
        return FocalLoss(alpha =alpha, gamma=gamma)
    elif criterion_choice == "DiceLoss":
        return DiceLoss()
    elif criterion_choice == "BCEWithLogitsLoss":
        return nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight])) if pos_weight else nn.BCEWithLogitsLoss()
    return nn.BCEWithLogitsLoss() 

Helper functions that could use tweakin to improve model performance

In [13]:
def FOLDS_GENERATOR(X, Y, normalisation_method=MinMaxScaler(), n_splits=5, random_state=None, oversampler=None):
    """
    Generates stratified folds with specified normalization.
    
    For list of scalers, see:
    https://scikit-learn.org/stable/api/sklearn.preprocessing.html
    
    For more details on scaling and normalization effects, see:
    https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#
    
    normalisation_method should be an instance of a scaler, e.g.,
    - MinMaxScaler()
    - MaxAbsScaler()
    - Quantile_Transform(output_distribution='uniform')
    
    Returns a list of tuples, each containing:
    (X_train_scaled, X_test_scaled, Y_train, Y_test), representing data for each fold
    """
    kF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    kFolds_list = []
    
    for fold, (train_idx, test_idx) in enumerate(kF.split(X, Y)):
        # Split the data into training and testing sets for this fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
        
        # IsolationForest for outlier removal (optional)
        iso_forest = IsolationForest(contamination=0.05, random_state=random_state)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)
            outliers = iso_forest.fit_predict(X_train)    
        X_train = X_train[outliers == 1]
        Y_train = Y_train[outliers == 1]
        
        # Scale the entire data (binary and continuous together)
        X_train_scaled = normalisation_method.fit_transform(X_train)
        X_test_scaled = normalisation_method.transform(X_test)

        # Handle oversampling if needed
        if oversampler:
            # Apply oversampling to both features and target
            X_train_scaled, Y_train = oversampler.fit_resample(X_train_scaled, Y_train)

        # Convert scaled data back to DataFrame with the correct column names
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
        
        # Handle community columns
        community_cols = [col for col in X_train_scaled.columns if col.startswith('Community')] 

        # Check for rows where multiple communities are flagged
        for idx, row in X_train_scaled[community_cols].iterrows():
            if set(np.unique(row)) != {0, 1}:  # If the unique values aren't just 0 or 1
                # Fix row by ensuring only one community is marked
                X_train_scaled.loc[idx, community_cols] = 0  # Set all community columns to 0
                max_col = row.idxmax()  # Find the column with the maximum value
                X_train_scaled.at[idx, max_col] = 1  # Set the column with the max value to 1

        # Ensure 'gender' is still binary (0 or 1)
        if 'Gender' in X_train_scaled.columns:
            X_train_scaled['Gender'] = (X_train_scaled['Gender'] > 0.5).astype(int)
            X_test_scaled['Gender'] = (X_test_scaled['Gender'] > 0.5).astype(int)
        
        # Append the processed fold to the list
        kFolds_list.append((X_train_scaled, X_test_scaled, Y_train, Y_test))
        
        print(f"Fold: {fold+1}, Train: {kFolds_list[fold][0].shape}, Test: {kFolds_list[fold][1].shape}")
    
    return kFolds_list


In [14]:
def train_and_evaluate(model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=20, patience=5, device=device):
    if isinstance(model.last_layer(), nn.Sigmoid) and isinstance(criterion, nn.BCEWithLogitsLoss):
        raise ValueError("Model output is Sigmoid but criterion is BCEWithLogitsLoss. Please check your model and criterion compatibility.")
    best_val_loss = float('inf')
    best_model_state = None
    wait = 0

    
    #* Epoch Training loop for this fold
    for epoch in range(1,epochs+1):
        #* Set model to training mode: essential for dropout and batch norm layers
        model.train()
        running_loss = 0.0 #? loss for this epoch
        #* Mini-batch training loop
        for batch, (inputs, labels) in enumerate(train_loader,start=1):
            optimiser.zero_grad() #? Zero the gradients
            outputs = model(inputs) #? Forward pass through the model
            loss = criterion(outputs, labels) #? Calculate loss
            loss.backward() #? Backpropagation
            running_loss += loss.item()
            optimiser.step() #? Update weights
            if isinstance(scheduler,torch.optim.lr_scheduler.OneCycleLR):
                scheduler.step()
        if not isinstance(scheduler,torch.optim.lr_scheduler.OneCycleLR):
            scheduler.step()
                
        train_loss = running_loss / len(train_loader)
        # print(f"Epoch: {epoch}, training loss: {train_loss:.4f}")
    
        #* Now we evaluate the model on the validation set, to track training vs validation loss
        model.eval() #? Set model to evaluation mode
        with torch.no_grad(): #? No need to track gradients during evaluation
            val_loss = 0.0    
            for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                outputs = model(inputs)
                # labels = labels.cpu() 
                loss = criterion(outputs, labels)
                val_loss += loss.item() #? Calculate loss
                avg_val_loss = val_loss / len(val_loader)
            # print(f"Epoch: {epoch}, training loss: {train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
            print(f"Epoch: {epoch}".ljust(12), f"training loss:{train_loss:.4f}".ljust(12), f"Val Loss: {avg_val_loss:.4f}",end="\r")
        
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            wait = 0
        else:
            if avg_val_loss*0.95 <= best_val_loss:
                wait += 1
            if wait >= patience:
                print(f"Early stopping triggered at epoch {epoch}, best val loss: {best_val_loss:.4f}")
                break
    
    #* Use best model to calculate metrics on the validation set
    #! must be outside epoch loop, it comes after the training and cv loop
    model.load_state_dict(best_model_state) #? Load the best model state
    with torch.no_grad():
        for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                labels = labels.cpu() 
                predictions = (torch.sigmoid(outputs) < 0.5).float().cpu().numpy()
                
                val_loss += loss.item() #? Calculate loss
                
    #! The following should have length equal to fold number           
    accuracy=accuracy_score(labels, predictions) 
    precision=precision_score(labels, predictions, pos_label=1, zero_division=0)
    recall=recall_score(labels, predictions, pos_label=1)
    f1=f1_score(labels, predictions, pos_label=1)
    auc=roc_auc_score(labels, predictions)
    
    return model, accuracy, precision, recall, f1, auc


In [15]:


class BinaryClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1),
            # nn.Sigmoid()
   
        )

    def forward(self, x):
        return self.net(x)
    
    def last_layer(self):
        return self.net[-1]

In [23]:
import torch
import torch.nn as nn

class FeedForwardBlock(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.1, activation=nn.ReLU):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.BatchNorm1d(out_features),
            activation(),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.block(x)


class MyModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout=0.2):
        super().__init__()
        
        # A couple of FeedForward blocks
        self.block1 = FeedForwardBlock(input_dim, hidden_dim, dropout)
        self.block2 = FeedForwardBlock(hidden_dim, hidden_dim, dropout)
        self.block3 = FeedForwardBlock(hidden_dim, output_dim, dropout)

        # Final output layer (could be softmax, sigmoid, or whatever your target is)
        self.output_layer = nn.Linear(output_dim, 1)  # Just in case you're doing regression or binary classification

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.output_layer(x)  # Final linear layer
        return x
    
    def last_layer(self):
        return self.output_layer


In [26]:
def maximise_combined_score(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    epochs = 1000
    random_state = 42
    oversampler = ADASYN(sampling_strategy='minority', n_neighbors=5, random_state=random_state)
    if isinstance(oversampler, ADASYN):
        n_neighbours = trial.suggest_int("n_neighbours", 1, 10)
        oversampler = ADASYN(sampling_strategy='minority', n_neighbors=n_neighbours, random_state=random_state)
    
    normalisation_method = trial.suggest_categorical("normalisation_method", [
        "MinMaxScaler",
        # "MaxAbsScaler",
        # "StandardScaler",
        # "RobustScaler",
        # "PowerTransformer",
        # "QuantileTransformer",
    ])
    if normalisation_method:
        if normalisation_method == "MinMaxScaler":
            normalisation_method = MinMaxScaler()
        elif normalisation_method == "MaxAbsScaler":
            normalisation_method = MaxAbsScaler()
        elif normalisation_method == "StandardScaler":
            normalisation_method = StandardScaler()
        elif normalisation_method == "RobustScaler":
            normalisation_method = RobustScaler()
        elif normalisation_method == "PowerTransformer":
            normalisation_method = PowerTransformer()
        elif normalisation_method == "QuantileTransformer":
            normalisation_method = QuantileTransformer(output_distribution='uniform')
        else:
            normalisation_method = MinMaxScaler()
    
    kFolds = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, 
                         normalisation_method = normalisation_method, 
                         n_splits=5, 
                         oversampler = oversampler, random_state=42)
                        
    # Model hyperparameters (first-level optimization)
    hidden_dim = trial.suggest_int("hidden_dim", 28, 256)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    initial_lr = trial.suggest_float("initial_lr", 1e-5, 1e-3, log=True)
    max_lr = trial.suggest_float("max_lr", 1e-3, 1e-1, log=True)
    
    # Loss function hyperparameters
    criterion_choice = trial.suggest_categorical("criterion", ["BCEWithLogitsLoss", "FocalLoss"]) 
    
    # Hyperparameter exploration optimization
    if criterion_choice == "BCEWithLogitsLoss":
        pos_weight = trial.suggest_int("pos_weight", 1, 10)
        alpha = None
        gamma = None
    elif criterion_choice == "FocalLoss":
        pos_weight= None
        alpha = trial.suggest_float("alpha", 1, 10)
        gamma = trial.suggest_float("gamma", 0, 5)
    else:
        pos_weight = None
    
    # Initialize lists for metrics across folds
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    auc_list = []

    # Cross-validation loop
    for fold, (train_x, test_x, train_y, test_y) in enumerate(kFolds, start=1):
        # Create DataLoader for current fold
        train_loader, val_loader = fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=256, device=device)
        # Calculate steps_per_epoch from the current fold's train_loader
        train_loader_len = len(train_loader)
        
        # Instantiate and initialize the model
        # model = BinaryClassifier(input_dim=get_feature_count(train_loader), hidden_dim=hidden_dim, dropout=dropout)
        model = MyModel(input_dim=get_feature_count(train_loader), hidden_dim=hidden_dim, output_dim=hidden_dim, dropout=dropout)
        model.to(device)
        model.apply(init_weights)
        
        # Map the choice to the actual loss function
        criterion = criterion_mapping(criterion_choice, pos_weight, alpha, gamma).to(device)
        optimiser = optim.Adam(model.parameters(), lr=initial_lr)

        
        # Initialize scheduler
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimiser,
            max_lr=max_lr,
            steps_per_epoch=train_loader_len,
            epochs=epochs,
            anneal_strategy='linear'
        )
        print(f"Fold {fold}:")
        # Train and evaluate the model on the current fold
        model, accuracy, precision, recall, f1, auc = train_and_evaluate(
            model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=epochs, patience=30, device=device
        )
        print(f"Accuracy: {accuracy:.4f}, precision: {precision:.4f}, recall: {recall:.4f}, f1: {f1:.4f}, auc: {auc:.4f}")

        # Append the metrics from the current fold
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        auc_list.append(auc)

    # Calculate the average metrics across all folds
    avg_accuracy = np.sum(accuracy_list) / len(accuracy_list)
    avg_precision = np.sum(precision_list) / len(precision_list)
    avg_recall = np.sum(recall_list) / len(recall_list)
    avg_f1 = np.sum(f1_list) / len(f1_list)
    avg_auc = np.sum(auc_list) / len(auc_list)

    # Combine metrics into a single "score"
    # combined_score = (avg_f1 + avg_precision + avg_recall + avg_accuracy + avg_auc) / 5
    combined_score = avg_f1

    return combined_score


In [27]:
import threading
import optuna
from optuna_dashboard import run_server

def start_dashboard():
    run_server(storage)

storage = optuna.storages.InMemoryStorage()
study = optuna.create_study(direction="maximize", storage=storage, study_name="Basic")

# Start dashboard in a separate thread
dashboard_thread = threading.Thread(target=start_dashboard, daemon=True)
dashboard_thread.start()

# Run optimization
study.optimize(maximise_combined_score, n_trials=30)

# After optimization, print results
print("Best trial:")
trial = study.best_trial
print(f"  Combined score: {trial.value}")
print("  Best hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-14 11:33:54,267] A new study created in memory with name: Basic
Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.



Using device: cuda
Fold: 1, Train: (8026, 28), Test: (1149, 28)
Fold: 2, Train: (7936, 28), Test: (1149, 28)
Fold: 3, Train: (7947, 28), Test: (1148, 28)
Fold: 4, Train: (7981, 28), Test: (1148, 28)
Fold: 5, Train: (7995, 28), Test: (1148, 28)
Fold 1:
Early stopping triggered at epoch 94, best val loss: 0.2759
Accuracy: 0.4143, precision: 0.0427, recall: 0.2241, f1: 0.0717, auc: 0.3299
Fold 2:
Early stopping triggered at epoch 83, best val loss: 0.2921
Accuracy: 0.4308, precision: 0.0410, recall: 0.2069, f1: 0.0684, auc: 0.3314
Fold 3:
Early stopping triggered at epoch 101, best val loss: 0.3028
Accuracy: 0.4538, precision: 0.0312, recall: 0.1466, f1: 0.0514, auc: 0.3175
Fold 4:


[W 2025-04-14 11:34:41,608] Trial 0 failed with parameters: {'n_neighbours': 9, 'normalisation_method': 'MinMaxScaler', 'hidden_dim': 142, 'dropout': 0.4201700667496542, 'initial_lr': 4.263934920119699e-05, 'max_lr': 0.0019540038822866484, 'criterion': 'FocalLoss', 'alpha': 2.8746306905206955, 'gamma': 2.7519792491813497} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "d:\GitHub repos\ADL\.venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\tanle\AppData\Local\Temp\ipykernel_30052\3458240061.py", line 96, in maximise_combined_score
    model, accuracy, precision, recall, f1, auc = train_and_evaluate(
                                                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\tanle\AppData\Local\Temp\ipykernel_30052\2150624751.py", line 20, in train_and_evaluate
    running_loss += loss.item()
                    ^^^^^^^^^

KeyboardInterrupt: 